# Secure System Medical Record with Blockchain System

## Initialize the role: 

| Role No | Role Name |Attributes| 
| :-----|:-----:| :-----|
| 1|NMPA | NMPA can approve and certify medicine incompatibility and to grant legal certification to doctors.|
| 2 |Patient | The Patient has the history of the illnesses, which are initiated with an empty list.
| 3|Doctor | The Doctor has the authorization which is by default eual to None and it takes a valid value when the NMPA sends him the authorization.|
| 4|Miner |  Each Miner has the complete history of the blockhchain and a wallet in which the fees are charged.|

## Initiate Blockchain

In [1]:
cd ../unique_blockchain

/Users/jennyjue/Documents/Secure_System_Medical_Record_With_Blockchain/unique_blockchain


In [2]:
#Load the relevant packages
import nacl.signing
from blockchain import Blockchain
from agents import Doctor, Miner, Patient, NMPA

When we start the blockchain we pass an object NMPA(National Medical Products Administration) , that the blockchain instantiates immediately. With the object NMPA we also want to pass a dictionary of medicines that the NMPA showed that are incompatible with some illnesses.

In [3]:
incompatibilities = {'medicine1': ['illness1', 'illness2'],
                    'medicine2': ['illness1'],
                    'medicine3': ['illness2', 'illness3']}

Health_block = Blockchain(NMPA, incompatibilities)

## Instantiate the agents

When we have the blockchain ready, we can think that the users start opening their account

In [4]:
# Miners
miner1 = Miner(Health_block)
miner2 = Miner(Health_block)
miner3 = Miner(Health_block)
miner4 = Miner(Health_block)
miner5 = Miner(Health_block)

# Doctors
doctor1 = Doctor('Doctor Tang')
doctor2 = Doctor('Doctor Song')
doctor3 = Doctor('Doctor Yuan')
doctor4 = Doctor('Doctor Ming')

# Patients
patient1 = Patient('Mr. Qin')
patient2 = Patient('Mr. Chu')
patient3 = Patient('Mr. Yan')
patient4 = Patient('Mr. Qi')
patient5 = Patient('Mr. Wei')
patient6 = Patient('Mr. Jin')

In [5]:
print('---------doctor---------')
print(doctor1.authorization)

print('---------patient---------')
print(patient1.illnesses)

print('---------miner---------')
print(miner1.chain)
print(miner1.wallet)

---------doctor---------
None
---------patient---------
[]
---------miner---------
[{'index': 1, 'timestamp': 1669098370.4895098, 'transactions': [], 'nounce': 100, 'previous_hash': 1}]
0


## Feature I：Incompatibility between medicines
### Role:NMPA , Behavior: Retrieve the incompatibilites or adding new ones

In [6]:
Health_block.NMPA.incompatibilities

{'medicine1': ['illness1', 'illness2'],
 'medicine2': ['illness1'],
 'medicine3': ['illness2', 'illness3']}

In [7]:
Health_block.NMPA.incompatibilities['medicine4'] =  ['illness2']

In [8]:
Health_block.NMPA.incompatibilities

{'medicine1': ['illness1', 'illness2'],
 'medicine2': ['illness1'],
 'medicine3': ['illness2', 'illness3'],
 'medicine4': ['illness2']}

## Feature Ⅱ：Authorization

###  Part Ⅰ: Role:Doctor , Behavior: No authorization

Now no doctor has an authorization, so let's see what happens if a doctor tries to do some diagnoses with no authorization

In [9]:
# We first create the 2 transactions
Health_block.new_diagnosis(doctor2, patient1, 'illness2', 0.20)
Health_block.new_diagnosis(doctor2, patient3, 'illness3', 0.20)

# Then we mine a block including all the miners
miners = [miner1, miner2, miner3, miner4]
Health_block.mine(miners)

Not valid Doctor authorization
Not valid Doctor authorization
Mined a new block with number 2
Block number 2 broadcasted to all the nodes


What happens is that every miner <font color = blue>reject the transaction </font>  because the authorization cannot be opened using the public key of the NMPA. So the block is mined, but<font color = blue> without transaction</font> . Let's inspect it.

In [10]:
Health_block.chain

[{'index': 1,
  'timestamp': 1669098370.4895098,
  'transactions': [],
  'nounce': 100,
  'previous_hash': 1},
 {'index': 2,
  'timestamp': 1669098525.512106,
  'transactions': [],
  'nounce': 226,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'}]

We can see that the second block does not have transactions and no illnessess have been appended to the history of the patients

In [11]:
print(patient1.illnesses)
print(patient3.illnesses)

[]
[]


### Part Ⅱ: Role: Doctor , Behavior: Authorization

Now let's create a new authorization by the NMPA for the doctor1

In [12]:
Health_block.new_authorization(doctor1, 0.50)
Health_block.mine(miners)

Mined a new block with number 3
Block number 3 broadcasted to all the nodes


In [13]:
Health_block.chain

[{'index': 1,
  'timestamp': 1669098370.4895098,
  'transactions': [],
  'nounce': 100,
  'previous_hash': 1},
 {'index': 2,
  'timestamp': 1669098525.512106,
  'transactions': [],
  'nounce': 226,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'},
 {'index': 3,
  'timestamp': 1669098593.379095,
  'transactions': [{'type': 'authorization',
    'sender': <agents.NMPA at 0x7fd508a6e730>,
    'recipient': <agents.Doctor at 0x7fd5188a0dc0>,
    'authorization': b'1\x1f1\xce4.]\xa1\x1c\xadF\xd2\xe5\x1c\x10\xb4\xb7\x13\xc0L\xaa\x8f\xe5\xe1b\xf4%G\x84\xcf\xd8\x9eLY?\xa9?\x8eP\xa0r\xa0>2\xfa\t\xeb7_Q\x11\xccY\xc0\\\x1b\xdd\x93G\xa5\xe7\xc3\xa0\x0356a2f0310bca9b94a5fcbfaf25b5d23cdf84d1faad05d402dd5aafcec9e77b7b',
    'fee': 0.5}],
  'nounce': 346,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'}]

In the transaction of the last block we now can see the authorization issued by the NMPA to the doctor. If we inspect the doctor authorization now we should see the unique code shown in the transaction

In [14]:
doctor1.authorization

b'1\x1f1\xce4.]\xa1\x1c\xadF\xd2\xe5\x1c\x10\xb4\xb7\x13\xc0L\xaa\x8f\xe5\xe1b\xf4%G\x84\xcf\xd8\x9eLY?\xa9?\x8eP\xa0r\xa0>2\xfa\t\xeb7_Q\x11\xccY\xc0\\\x1b\xdd\x93G\xa5\xe7\xc3\xa0\x0356a2f0310bca9b94a5fcbfaf25b5d23cdf84d1faad05d402dd5aafcec9e77b7b'

If doctor1 tries now to do diagnoses, these will have immediate effects on patients

In [15]:
Health_block.new_diagnosis(doctor1, patient3, 'illness1', 0.20)
Health_block.mine(miners)

Mined a new block with number 4
Block number 4 broadcasted to all the nodes


In [16]:
patient3.illnesses

['illness1']

### Part Ⅲ: Role:Doctor , Behavior: Fake Authorization

Let's see what happens if a doctor, say doctor2, creates a fake authorization and put it in its own variable authorization

In [17]:
# We first create a random privte key
private_key_random = nacl.signing.SigningKey.generate()

# We then sign the address of the doctor with that key
signed = private_key_random.sign(bytes(doctor2.address, 'utf-8'))

# Finally we insert the authorization in the variable
doctor2.authorization = signed

In [18]:
print(doctor2.authorization)

b'\x90\xa1\x0c\xac\x03\x15^\x941\x9a\x91[\xc2\xc84\x94{\x12E\xaa\xba\xd4hLLA\xcc\\\x7fhs\xf8h\x05\x84\x99|"R:\xd3;\x89\xc4d\xcf\x0cG;.\xb9\xd3l\xe3W\xe6\xc5\xc4-a\x8er\x85\x0f9222e9623fc4d671788146ed58b52d52b161e0f897cf7ce351d6025d96ad982c'


In [19]:
Health_block.new_diagnosis(doctor2, patient1, 'illness2', 0.20)
Health_block.mine(miners)

Not valid Doctor authorization
Mined a new block with number 5
Block number 5 broadcasted to all the nodes


Other miners detect the fake authorization and <mark>the last block will not contain not valid transactions<mark>

In [21]:
Health_block.chain

[{'index': 1,
  'timestamp': 1669098370.4895098,
  'transactions': [],
  'nounce': 100,
  'previous_hash': 1},
 {'index': 2,
  'timestamp': 1669098525.512106,
  'transactions': [],
  'nounce': 226,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'},
 {'index': 3,
  'timestamp': 1669098593.379095,
  'transactions': [{'type': 'authorization',
    'sender': <agents.NMPA at 0x7fd508a6e730>,
    'recipient': <agents.Doctor at 0x7fd5188a0dc0>,
    'authorization': b'1\x1f1\xce4.]\xa1\x1c\xadF\xd2\xe5\x1c\x10\xb4\xb7\x13\xc0L\xaa\x8f\xe5\xe1b\xf4%G\x84\xcf\xd8\x9eLY?\xa9?\x8eP\xa0r\xa0>2\xfa\t\xeb7_Q\x11\xccY\xc0\\\x1b\xdd\x93G\xa5\xe7\xc3\xa0\x0356a2f0310bca9b94a5fcbfaf25b5d23cdf84d1faad05d402dd5aafcec9e77b7b',
    'fee': 0.5}],
  'nounce': 346,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'},
 {'index': 4,
  'timestamp': 1669098661.172205,
  'transactions': [{'type': 'diagnosis',
    'sender': <agents.Doctor at 0x7fd

## Feature Ⅲ：Prescription

Another feature of the blockchain is that<font color = blue> prevents doctors to do prescriptions of medicines not compatible with the illness history of the patient</font> .

In [22]:
Health_block.NMPA.incompatibilities

{'medicine1': ['illness1', 'illness2'],
 'medicine2': ['illness1'],
 'medicine3': ['illness2', 'illness3'],
 'medicine4': ['illness2']}

In [23]:
patient3.illnesses

['illness1']

if we see above, patient3 had <font color = red>illness1</font> and looking at the incompatibilites declared by the NMPA  <font color = red>medicine1 </font>and <font color = red>medicine2 </font>are not compatible with this illness.

In [24]:
# Let's try to do a prescription incompatible with the illnesses of patient3
Health_block.new_prescription(doctor1, patient3, 'medicine2', 0.20)
Health_block.mine(miners)

Incompatibility of one prescription with the history of the patient
Mined a new block with number 6
Block number 6 broadcasted to all the nodes


Also in this case, the mined block <font color = blue>will not contain not allowed transaction as in this case</font> 

## Feature Ⅳ：Miner Wallet

Every Miner has a wallet in which the fees of the transactions are charged. Let's inspect the wallets of the miners at this point

In [25]:
print(miner1.wallet, miner2.wallet, miner3.wallet, miner4.wallet)

0.7 0 0 0


## Feature Ⅴ：Reading Permission ——Protecting Patient Privacy

The last feature that the blockchain implements is a temporary couple of private and public keys for each patient, so that the doctor can access the history of the patient only one time without future access to the patient's data 

In [26]:
# Let's first store the orivate key fo the patient
private_key = patient3.private_key

# Now let's access the history of the patient using the private key
print(Health_block.get_patient_history(patient3, private_key=private_key))

Temporary keys have been refreshed. Old keys have been destroyed and will not work.
['illness1']


If now we attempt to <font color=red> reaccess </font> the history again, this will be <mark>impossible

In [27]:
# If now we attempt to reaccess the history again, this will not be possible
print(Health_block.get_patient_history(patient3, private_key=private_key))

PermissionError: You do not have access to the BlockChain of this patient.

### After presenting the features, the scene simulation test is carried out next.